### 1. Cloning the Dataset

In [ ]:
!rm -r /content/OpenLibrary-Books-Scraper

!git clone https://github.com/AshishJangra27/OpenLibrary-Books-Scraper

!rm -r data
!mkdir data

rm: cannot remove '/content/OpenLibrary-Books-Scraper': No such file or directory
Cloning into 'OpenLibrary-Books-Scraper'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 119 (delta 13), reused 16 (delta 6), pack-reused 88
Receiving objects: 100% (119/119), 88.01 MiB | 24.93 MiB/s, done.
Resolving deltas: 100% (24/24), done.
rm: cannot remove 'data': No such file or directory


### 2. Importing Libraries

In [ ]:
import os
import requests
import numpy as np
import pandas as pd
from tqdm import tqdm
import multiprocessing
from bs4 import BeautifulSoup
from google.colab import drive

for i in os.listdir('/content/OpenLibrary-Books-Scraper/data'):
  os.system('gunzip /content/OpenLibrary-Books-Scraper/data/' + i)

!python /content/OpenLibrary-Books-Scraper/app.py

df = pd.read_csv('data.csv').drop_duplicates('title_id')

del df['category_']

df['book_stats'] = 'Nan'
df['descriptions'] = 'Nan'
df['reading_stats'] = 'Nan'

drive.mount('/content/gdrive/', force_remount=True)

100% 57/57 [00:18<00:00,  3.08it/s]
Mounted at /content/gdrive/


In [ ]:
def download(st,en):

  book_stats = []
  descriptions = []
  reading_stats = []

  for ids in tqdm(df['title_id'][st:en]):
    link = 'https://openlibrary.org/' + ids
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')

    try:
      reading_stat = '|'.join([ i.text.replace('\n',' ').strip() for i in soup.find('div', class_ = 'readers-stats__wrapper mobile').find_all('li', class_ = 'reading-log-stat')])
    except:
      reading_stat = np.nan
    try:
      book_stat = '|'.join([ i.text.replace('\n',' ').strip() for i in soup.find('div', class_ = 'edition-omniline').find_all('div', class_ = 'edition-omniline-item')])
    except:
      book_stat = np.nan
    try:
      description = soup.find('div', class_ = 'book-description').text.replace('\n',' ').strip()
    except:
      description = np.nan

    reading_stats.append(reading_stat)
    book_stats.append(book_stat)
    descriptions.append(description)

  df['book_stats'][st:en]    = book_stats
  df['descriptions'][st:en]  = descriptions
  df['reading_stats'][st:en] = reading_stats

  df[st:en].to_csv('data/' + str(st) + '_' + str(en) + '.csv', index = False)

In [ ]:
for start in range(1270000,1350000,10000):

  n = 2000
  pr = 5
  process = []
  for i in range(start, start + n*pr,n):
    process.append(multiprocessing.Process(target = download, args = ( i , i + n)))
  for p in process:
    p.start()
  for p in process:
    p.join()

  csvs = [csv for csv in os.listdir('data') if '.csv' in csv]
  data = pd.DataFrame()
  for csv in tqdm(csvs):
      df_ = pd.read_csv('data/' + csv )
      data = pd.concat((data,df_))

  data.to_csv('/content/gdrive/MyDrive/Book Dataset/'+str(start)+'_'+str(start + pr * n)+'.csv' , index = False)

  !rm -r data
  !mkdir data

 79%|███████▉  | 1579/2000 [28:58<07:43,  1.10s/it]
Process Process-2:
Traceback (most recent call last):
  File "/usr/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/usr/lib/python3.10/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "<ipython-input-3-53929ddf5fa2>", line 9, in download
    r = requests.get(link)
  File "/usr/local/lib/python3.10/dist-packages/requests/api.py", line 75, in get
    return request('get', url, params=params, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/api.py", line 61, in request
    return session.request(method=method, url=url, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 529, in request
    resp = self.send(prep, **send_kwargs)
  File "/usr/local/lib/python3.10/dist-packages/requests/sessions.py", line 667, in send
    history = [resp for resp in gen]
  File "/usr/local/lib/python3.10/dist-packag